# cosmoprimo bindings
cosmoprimo comes with bindings for [Cobaya](https://github.com/CobayaSampler/cobaya) and [CosmoSIS](https://github.com/joezuntz/cosmosis).
This notebook requires Cobaya and CosmoSIS to be installed.

## Cobaya
For Cobaya, just provide the theory.

In [1]:
from cosmoprimo.fiducial import DESI
cosmo = DESI()

for engine in ['class', 'camb', 'isitgr']:

    params = {'Omega_m': {'prior': {'min': 0.1, 'max': 1.},
              'ref': {'dist': 'norm', 'loc': 0.3, 'scale': 0.01},
              'latex': '\Omega_{m}'},
              'omega_b': cosmo['omega_b'],
              'H0': cosmo['H0'],
              'A_s': cosmo['A_s'],
              'n_s': cosmo['n_s'],
              'tau_reio': cosmo['tau_reio']}

    extra_args = {'m_ncdm': cosmo['m_ncdm'], 'N_ur': cosmo['N_ur']}

    if engine == 'isitgr':
        params['Q1'] = 1.5
        params['Q2'] = 0.5
        extra_args.update(parameterization='QD', binning='hybrid', z_div=1, z_TGR=2, z_tw=0.05,
                          k_c=0.01, Q3=1.5, Q4=0.5, D1=1, D2=1, D3=1, D4=1)
    
    info = {'params': params,
            'likelihood': {'sn.pantheon': None, 'H0.riess2020': None, 'bao.sdss_dr12_consensus_final': None, 'planck_2018_highl_plik.TTTEEE': None},
            # This is the cosmoprimo theory, extra_args is passed to cosmoprimo.Cosmology()
            'theory': {'cosmoprimo.bindings.cobaya.cosmoprimo': {'engine': engine, 'stop_at_error': True, 'extra_args': extra_args}}}
    info_sampler = {'evaluate': {}}

    from cobaya.model import get_model
    from cobaya.sampler import get_sampler
    model = get_model(info)
    get_sampler(info_sampler, model=model).run()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[prior] *WARNING* External prior 'SZ' loaded. Mind that it might not be normalized!
[cosmoprimo.bindings.cobaya.cosmoprimo] `cosmoprimo` module loaded successfully from /home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo
[bao.sdss_dr12_consensus_final] Initialized.
[planck_2018_highl_plik.ttteee] `clik` module loaded successfully from /home/adematti/Bureau/DESI/NERSC/lib/COM_Likelihood_Code-v3.0_R3.10/code/plc_3.0/plc-3.1/lib/python/site-packages/clik
[prior] *WARNING* There are unbounded parameters (['A_planck', 'calib_100T', 'calib_217T', 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 'galf_TE_A_217']). Prior bounds are given at 0.9999995 confidence level. Beware of likelihood modes at the edge of the prior
[evaluate] Initialized!
[evaluate] Looking for a reference point with non-zero prior.
[evaluate] Reference point:
   Omega_m = 0.301248
   A_planck = 

## CosmoSIS
For CosmoSIS, use the module.

In [2]:
import os
# We need the path to cosmoprimo...
import cosmoprimo
os.environ['COSMOPRIMO_DIR'] = os.path.dirname(os.path.dirname(cosmoprimo.__file__))
# Also, if not defined, provide path to cosmosis-standard-library:
#os.environ['COSMOSIS_STD_DIR'] = 

In [3]:
!mkdir -p _tests

/bin/bash: /home/adematti/anaconda3/envs/cosmodesi-main/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
%%file _tests/cosmosis_config.ini

[DEFAULT]
fatal_errors = T
planck_path = ${PLANCK_SRC_DIR}/baseline/plc_3.0

[runtime]
sampler = test

[output]
filename = _tests/test.txt
format = text
verbosity = 0

[pipeline]
modules = consistency cosmoprimo pantheon planck
values = _tests/cosmosis_values.ini
likelihoods = pantheon planck2018
quiet = T
debug = F
timing = F

[consistency]
file = ${COSMOSIS_STD_DIR}/utility/consistency/consistency_interface.py

[cosmoprimo]
file = ${COSMOPRIMO_DIR}/cosmoprimo/bindings/cosmosis/cosmoprimo_interface.py
lmax = 2800
lensing = True
engine = class

[pantheon]
file = ${COSMOSIS_STD_DIR}/likelihood/pantheon/pantheon.py

[planck]
;Planck 2018 high ell TT,TE and EE + low ell TT + low ell EE (in Planck notations = TT+lowE)
;without CMB lensing
file = ${COSMOSIS_STD_DIR}/likelihood/planck2018/planck_interface.so
data_1 = %(planck_path)s/hi_l/plik/plik_rd12_HM_v22b_TTTEEE.clik
data_2 = %(planck_path)s/low_l/commander/commander_dx12_v3_2_29.clik
data_3 = %(planck_path)s/low_l/simall/simall_100x143_offlike5_EE_Aplanck_B.clik

[test]

Overwriting _tests/cosmosis_config.ini


In [5]:
%%file _tests/cosmosis_values.ini

[cosmological_parameters]
omega_m = 0.1 0.3 0.9
ombh2 = 0.02237
h0 = 0.6736
A_s = 2.083e-09
n_s = 0.9649
tau = 0.0544

mnu = 0.06
nnu = 3.046
num_massive_neutrinos = 1
omega_k = 0.0
w = -1.0
wa = 0.0

[supernova_params]
M = -19.263

[planck]
a_planck          = 0.9 1.0 1.1 ; Total Planck calibration (relative to 1) at map level, scales all channels
;adapted from cosmomc where param[paramname] = center min max start_width propose_width
;comments for parameters are taken from cosmomc file
a_cib_217         = 0. 67. 200. ;Spectral index of the CIB
cib_index         = -1.3        ;Spectral index of the CIB
xi_sz_cib         = 0. 0. 1.    ;TSZ-CIB template amplitude (positive is negative)
a_sz              = 0. 7. 10.   ;Thermal SZ amplitude at 143 GH
ps_a_100_100      = 0. 257. 400. ;Point source amplitude at 100 GHz
ps_a_143_143      = 0. 47. 400. ;Point source amplitude at 143 GHz
ps_a_143_217      = 0. 40. 400. ;Point source amplitude at 143x217 GHz
ps_a_217_217      = 0. 104. 400. ;Point source amplitude at 217 GHz
ksz_norm          = 0. 0. 10. ;Kinetic SZ amplitude at 143 GHz
gal545_a_100      = 0. 7. 50. ;Dust amplitude at 100 GHz
gal545_a_143      = 0. 9. 50. ;Dust amplitude at 143 GHz
gal545_a_143_217  = 0. 21. 100. ;Dust amplitude at 143x217 GHz
gal545_a_217      = 0. 80. 400. ;Dust amplitude at 217 GHz
a_sbpx_100_100_tt = 1. ;Subpix noise multiplicative factor at 100 GHz
a_sbpx_143_143_tt = 1. ;Subpix noise multiplicative factor at 143 GHz
a_sbpx_143_217_tt = 1. ;Subpix noise multiplicative factor at 143x217 GHz
a_sbpx_217_217_tt = 1. ;Subpix noise multiplicative factor at 217 GHz
calib_100t        = 0. 1.0002 3. ;Relative power spectrum calibration factor 100/143
calib_217t        = 0. 0.99805 3. ;Relative power spectrum calibration factor 217/143
calib_100p        = 1.021
calib_143p        = 0.966
calib_217p        = 1.04
galf_ee_a_100     = 0.055
galf_ee_a_100_143 = 0.040
galf_ee_a_100_217 = 0.094
galf_ee_a_143     = 0.086
galf_ee_a_143_217 = 0.21
galf_ee_a_217     = 0.70
galf_ee_index     = -2.4
galf_te_a_100     = 0 0.13 10
galf_te_a_100_143 = 0 0.13 10
galf_te_a_100_217 = 0 0.46 10
galf_te_a_143     = 0 0.207 10
galf_te_a_143_217 = 0 0.69 10
galf_te_a_217     = 0 1.938 10
galf_te_index     = -2.4
a_cnoise_e2e_100_100_ee = 1.
a_cnoise_e2e_143_143_ee = 1.
a_cnoise_e2e_217_217_ee = 1.
a_sbpx_100_100_ee       = 1.
a_sbpx_100_143_ee       = 1.
a_sbpx_100_217_ee       = 1.
a_sbpx_143_143_ee       = 1.
a_sbpx_143_217_ee       = 1.
a_sbpx_217_217_ee       = 1.
a_pol                   = 1.

Overwriting _tests/cosmosis_values.ini


In [6]:
!cosmosis _tests/cosmosis_config.ini

/bin/bash: /home/adematti/anaconda3/envs/cosmodesi-main/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Deprecated: The [pipeline] quiet option is deprecated.  Set [runtime] verbosity instead.
Setting up pipeline from parameter file _tests/cosmosis_config.ini
------------------------------------------------------------------

Setting up module consistency
------------------------------

Setting up module cosmoprimo
-----------------------------
**** WARNING: Parameter 'engine' in the [cosmoprimo] section never used!


Setting up module pantheon
---------------------------
Loading Pantheon data from /home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmosis-standard-library//likelihood/pantheon/lcparam_DS17f.txt
Found 40 Pantheon supernovae (or bins if you used the binned data file)
Loading Pantheon covariance from /home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmosis-standard-library//likelihood/pantheon/sys_DS17f.txt

Setting up module planck
-----------------------